![pipeline](pictures/pictures.002.png)

# Text-Fabric from ETCBC

This notebook assembles the data from the ETCBC that is needed
to compile its datasets in text-fabric-format on GitHub.
Ultimately the data for the website [SHEBANQ](https://shebanq.ancient-data.org) will be
derived from these Text-Fabric-sources.

## Pipeline
This is **pipe 1** of the pipeline from ETCBC data to the website SHEBANQ.

A run of this pipe produces a data *version*.
It should be run whenever there are new or updated data sources present that affect the output data.
Since all input data is delivered in a GitHub repo, we have excellent machinery to 
work with versioning.

The pipe works by executing a series of programs, contained in GitHub repositories.
For each repository in the pipe, a series of notebooks will be executed.
See [script mode](https://github.com/ETCBC/pipeline/blob/master/README.md#operation) for 
details on how we call notebooks.

All this is specified in the configuration below.

### Core data

The core data is delivered by the ETCBC as `bhsa.mql.bz2` in 
the GitHub repo [bhsa](https://github.com/ETCBC/bhsa) in directory `source`.

This data will be converted by `tfFromMQL` in the `programs` directory.

The result of this action will be an updated Text-Fabric resource in its 
`tf/core` directory.

### Additional data

Researchers have contributed to the dataset, 
but not all that data is in the core.
They are typically in the repository where the research has been 
executed, and where the data is documented.

Before the pipe starts, these repos must be pulled.

## Continuous version
Version `c` acts as a *continuous* version. It will be overwritten
by new snapshots of the data on a regular basis.

We support the following workflow to carry out these updates:

1. make a new version called `_temp`. Note:
   * this choice of name prevents it to reach github, because `_temp` directories are in `.gitignore`;
   * after running this workflow, the version `_temp` already exists, this is not a problem;
2. put a new data snapshot in the `source/_temp` directory of the `bhsa` repo, add also data to the
   `source/_temp` directories of the other repos in the pipeline, as far as relevant;
3. run `good = runPipeline(pipeline, versions=['_temp'], force=True)`. Note:
   * we use `force=True` here, because then the old data in version `_temp` will be thoroughly overwritten;
4. if all went well run `copyVersion(pipeline, '_temp', 'c')`. This will overwrite all data directories
   in version `c` by the just created data directories in `temp`.

In [1]:
import os,sys,collections
from pipeline import runPipeline, copyVersion
from tf.fabric import Fabric

# Config

In [2]:
CORE_NAME = 'bhsa'

if 'SCRIPT' not in locals(): 
    SCRIPT = False
    DEFAULT_CORE_NAME = CORE_NAME
    DEFAULT_VERSION = 'c'

# Pipeline settings

Here all the nitty-gritty differences between versions are stated.

In [3]:
pipeline = dict(
    defaults = dict(
        CORE_NAME=CORE_NAME,
        VERSION=DEFAULT_VERSION,
        LANG_FEATURE='language',
        OCC_FEATURE='g_cons',
        LEX_FEATURE='lex',
        TEXT_FEATURE='g_word_utf8',
        TRAILER_FEATURE='trailer_utf8',
        DO_VOCALIZED_LEXEME=True,
        EXTRA_OVERLAP='',
        LEX_FORMATS='@fmt:lex-trans-plain={lex0} ',
        RENAME=(
            ('g_suffix', 'trailer'),
            ('g_suffix_utf8', 'trailer_utf8'),
        ),
    ),
    versions={
        '_temp': dict(),
        'c': dict(),
        '2017': dict(),
        '2016': dict(),
        '4b': dict(
                DO_VOCALIZED_LEXEME=False,
                EXTRA_OVERLAP='gloss nametype',
                LEX_FORMATS='@fmt:lex-trans-plain={lex} ',
            ),
        '4': dict(
                DO_VOCALIZED_LEXEME=False,
                EXTRA_OVERLAP='gloss nametype',
                LEX_FORMATS='@fmt:lex-trans-plain={lex} ',
        ),
        '3': dict(
                LANG_FEATURE='language',
                OCC_FEATURE='surface_consonants',
                LEX_FEATURE='lexeme',
                TEXT_FEATURE='text',
                TRAILER_FEATURE='suffix',
                LEX_FORMAT='@fmt:lex-trans-plain={lexeme} ',
            ),
    },
    repoOrder = '''
        bhsa
        phono
        valence
        parallels
    ''',
    repoConfig = dict(
        bhsa=(
            dict(
                task='coreData',
            ),
            dict(
                task='bookNames',
                omit={},
            ),
            dict(
                task='lexicon',
                omit={'3'},
            ),
            dict(
                task='paragraphs',
                 omit={'3', '4', '4b'},
            ),
            dict(
                task='ketivQere',
                omit={'3', '4', '4b'},
            ),
            dict(
                task='stats',
                omit={'4', '4b'},
            ),
        ),
        phono=(
            dict(
                task='phono',
                omit={'3', '4', '4b'},
            ),
        ),
        valence=(
            dict(
                task='enrich',
                omit={'3'},
            ),
            dict(
                task='flowchart',
                omit={'3'},
            ),
        ),
        parallels=(
            dict(
                task='parallels',
                omit={},
                params=dict(
                    FORCE_MATRIX=False,
                ),
            ),
        ),
    ),
    repoDataDirs = dict(
        bhsa      = 'source _temp tf shebanq',
        phono     = '_temp tf',
        valence   = 'source _temp tf shebanq',
        parallels = 'source _temp tf',
    ),
)

# Run the pipeline

To run everything from scratch:

```python
good = runPipeline(pipeline, versions=['3', '4', '4b', '2016', '2017', 'c'], force=True)
```

To make a new version called `temp`

```python
good = runPipeline(pipeline, versions=['_temp'], force=True) # a new candidate for 'c'
```

To copy the `_temp` version over to the continuous version `c`:

```python
if good:
    copyVersion(pipeline, '_temp', 'c')
```
This will copy all data, `source`, `_temp` and `tf` over from `_temp` to `c`.

If you want to create the node mappings between versions,
go to the versionMappings notebook in the BHSA repo and run it.

In [9]:
# good = runPipeline(pipeline, versions=['3', '4', '4b', '2016', '2017', 'c'], force=True)
good = runPipeline(pipeline, versions=['2016', '2017', 'c'], force=True)


##############################################################################################
#                                                                                            #
#      9m 53s Make version [2016]                                                            #
#                                                                                            #
##############################################################################################


**********************************************************************************************
*                                                                                            *
*      9m 53s Make repo [bhsa]                                                               *
*                                                                                            *
**********************************************************************************************


---------------------------------------------

  0.28s 		otype half_verse
  0.28s 			feature label (str) =def=  : node
  0.29s 		otype verse
  0.29s 			feature verse (int) =def= 0 : node
  0.29s 			feature chapter (int) =def= 0 : node
  0.29s 			feature label (str) =def=  : node
  0.29s 			feature book (str) =def= Genesis : node
  0.30s 		otype phrase_atom
  0.30s 			feature number (int) =def= 0 : node
  0.30s 			feature dist (int) =def= 0 : node
  0.30s 			feature distributional_parent (str) =def= 0 : edge
  0.31s 			feature mother (str) =def= 0 : edge
  0.31s 			feature functional_parent (str) =def= 0 : edge
  0.31s 			feature det (str) =def= NA : node
  0.31s 			feature typ (str) =def= VP : node
  0.31s 			feature rela (str) =def= NA : node
  0.31s 			feature dist_unit (str) =def= clause_atoms : node
  0.32s 		objects in word
  7.35s 	line   1000000
    14s 	line   2000000
    18s 		objects in word
    20s 	line   3000000
    26s 	line   4000000
    32s 	line   5000000
    34s 		objects in word
    39s 	line   6000000
    45s 	l

   |     0.83s T prs_ps               to /Users/dirk/github/etcbc/bhsa/_temp/2016/tf
   |     0.74s T ps                   to /Users/dirk/github/etcbc/bhsa/_temp/2016/tf
   |     0.76s T qere                 to /Users/dirk/github/etcbc/bhsa/_temp/2016/tf
   |     0.81s T qere_utf8            to /Users/dirk/github/etcbc/bhsa/_temp/2016/tf
   |     1.35s T rela                 to /Users/dirk/github/etcbc/bhsa/_temp/2016/tf
   |     0.80s T sp                   to /Users/dirk/github/etcbc/bhsa/_temp/2016/tf
   |     0.75s T st                   to /Users/dirk/github/etcbc/bhsa/_temp/2016/tf
   |     0.17s T tab                  to /Users/dirk/github/etcbc/bhsa/_temp/2016/tf
   |     0.80s T trailer              to /Users/dirk/github/etcbc/bhsa/_temp/2016/tf
   |     0.98s T trailer_utf8         to /Users/dirk/github/etcbc/bhsa/_temp/2016/tf
   |     0.18s T txt                  to /Users/dirk/github/etcbc/bhsa/_temp/2016/tf
   |     1.28s T typ                  to /Users/dirk/github/etcbc

|     14m 53s pdp                       ... no changes
|     14m 53s pfm                       ... no changes
|     14m 54s prs                       ... no changes
|     14m 54s prs_gn                    ... no changes
|     14m 54s prs_nu                    ... no changes
|     14m 55s prs_ps                    ... no changes
|     14m 55s ps                        ... no changes
|     14m 56s qere                      ... differences after the metadata
|     14m 56s 	line      2 OLD -->3897	HAJ:Y;74><--
|     14m 56s 	line      2 NEW --><--
|     14m 56s 	line      3 OLD -->4420	>@H:@LO75W<--
|     14m 56s 	line      3 NEW --><--
|     14m 56s 	line      4 OLD -->5645	>@H:@LO92W<--
|     14m 56s 	line      4 NEW --><--
|     14m 56s 	line      5 OLD -->5912	>@95H:@LOW03<--
|     14m 56s 	line      5 NEW --><--

|     14m 56s qere_utf8                 ... differences after the metadata
|     14m 56s 	line      2 OLD -->3897	הַיְצֵ֣א<--
|     14m 56s 	line      2 NEW --><--
|     14m 

   |     1.52s T nu                   from /Users/dirk/github/etcbc/bhsa/tf/2016
   |     2.45s T number               from /Users/dirk/github/etcbc/bhsa/tf/2016
   |     1.45s T pdp                  from /Users/dirk/github/etcbc/bhsa/tf/2016
   |     1.48s T pfm                  from /Users/dirk/github/etcbc/bhsa/tf/2016
   |     1.51s T prs                  from /Users/dirk/github/etcbc/bhsa/tf/2016
   |     1.57s T prs_gn               from /Users/dirk/github/etcbc/bhsa/tf/2016
   |     1.70s T prs_nu               from /Users/dirk/github/etcbc/bhsa/tf/2016
   |     1.60s T prs_ps               from /Users/dirk/github/etcbc/bhsa/tf/2016
   |     1.53s T ps                   from /Users/dirk/github/etcbc/bhsa/tf/2016
   |     0.71s T qere                 from /Users/dirk/github/etcbc/bhsa/tf/2016
   |     0.72s T qere_utf8            from /Users/dirk/github/etcbc/bhsa/tf/2016
   |     2.62s T rela                 from /Users/dirk/github/etcbc/bhsa/tf/2016
   |     1.45s T sp         

...

|     53m 58s 	Destination /Users/dirk/github/etcbc/valence/tf/c/.tf/valence.tfx exists
|     53m 58s NOTE: repo seems up to date. Will be run because of "force=True"
True True
..............................................................................................
.     53m 58s Load the existing TF dataset                                                   .
..............................................................................................
This is Text-Fabric 3.1.1
Api reference : https://github.com/Dans-labs/text-fabric/wiki/Api
Tutorial      : https://github.com/Dans-labs/text-fabric/blob/master/docs/tutorial.ipynb
Example data  : https://github.com/Dans-labs/text-fabric-data

114 features found and 0 ignored
  0.00s loading features ...
   |     0.21s B lex_utf8             from /Users/dirk/github/etcbc/bhsa/tf/c
   |     0.13s B lex                  from /Users/dirk/github/etcbc/bhsa/tf/c
   |     0.01s B gloss                from /Users/dirk/github/etcbc/bhsa/tf/

|     54m 18s 	Done
|     54m 18s 	Phrases of kind C :  19302
|     54m 18s 	Phrases of kind L :  11680
|     54m 18s 	Phrases of kind I :   6018
|     54m 18s 	Total complements :  37000
|     54m 18s 	Total phrases     : 214541
..............................................................................................
.     54m 18s Checking enrichment logic                                                      .
..............................................................................................
|     54m 18s 	All 6 rules OK
..............................................................................................
.     54m 18s Generating enrichments                                                         .
..............................................................................................
|     54m 24s 	Generated enrichment values for 1380 verbs:
|     54m 24s 	Enriched values for 221366 nodes
|     54m 24s 	Overview of rule applications:
|     54m 24s gen

   |     0.25s B typ                  from /Users/dirk/github/etcbc/bhsa/tf/c
   |     0.08s B function             from /Users/dirk/github/etcbc/bhsa/tf/c
   |     0.83s T valence              from /Users/dirk/github/etcbc/valence/tf/c
   |     0.80s T predication          from /Users/dirk/github/etcbc/valence/tf/c
   |     0.76s T grammatical          from /Users/dirk/github/etcbc/valence/tf/c
   |     0.38s T original             from /Users/dirk/github/etcbc/valence/tf/c
   |     0.56s T lexical              from /Users/dirk/github/etcbc/valence/tf/c
   |     0.59s T semantic             from /Users/dirk/github/etcbc/valence/tf/c
   |     0.42s T f_correction         from /Users/dirk/github/etcbc/valence/tf/c
   |     0.38s T s_manual             from /Users/dirk/github/etcbc/valence/tf/c
   |     0.47s T cfunction            from /Users/dirk/github/etcbc/valence/tf/c
   |     0.00s Feature overview: 119 for nodes; 4 for edges; 1 configs; 7 computed
    11s All features loaded/comp

|     55m 07s 	    3 clauses with  3 cpls       constituents
|     55m 07s 	   87 clauses with  2 cpls       constituents
|     55m 07s 	 8696 clauses with  1 cpls       constituents
|     55m 07s 	38789 clauses with  0 cpls       constituents
|     55m 07s 	 8786 clauses with  a cpls       constituent
|     55m 07s 	    2 clauses with  2 bens       constituents
|     55m 07s 	  171 clauses with  1 bens       constituents
|     55m 07s 	47402 clauses with  0 bens       constituents
|     55m 07s 	  173 clauses with  a bens       constituent
|     55m 07s 	69449 clauses
..............................................................................................
.     55m 07s Checking the flowcharts                                                        .
..............................................................................................
|     55m 07s 	No flowchart for 1543 verbs, e.g. <BC, <BD, <BH, <BR, <BR=, <BT, <BV, <BV=, <CC, <CN
|     55m 07s 	All flowcharts belong to

   |     0.26s B number               from /Users/dirk/github/etcbc/bhsa/tf/c
   |     0.00s Feature overview: 109 for nodes; 4 for edges; 1 configs; 7 computed
  5.48s All features loaded/computed - for details use loadLog()
..............................................................................................
.     55m 50s CROSSREFS: Fetching crossrefs                                                  .
..............................................................................................
|     55m 50s 	Reading existing /Users/dirk/github/etcbc/parallels/_temp/parallelTable.tsv
|     55m 50s 		INFO: All verse nodes are the same as in the previous version
..............................................................................................
.     55m 50s Writing TF parallel features                                                   .
..............................................................................................
   |     0.07s T crossref          

# Consolidate the continuous version
If you have run an update version called `_temp`, and all has went well
you can copy over the entire version (including its source and temp directories to `c`).

In [ ]:
#good = True
#good = False

if good:
    copyVersion(pipeline, '_temp', 'c')